In [8]:
import numpy as np
import pandas as pd
import geopandas as gp
import shapely
import os

In [10]:
bay_county_names = ["Alameda", "ContraCosta", "Sonoma", "Solano", "SanMateo", "SantaClara", "SanFrancisco", "Marin","Napa"] 
analysis_years = ["2008", "2009", "2010", "2011", "2012", "2013", "2014", "2015", "2016", "2017"] #try to use 2010 census for this



slump_years = ["2008", "2009", "2010", "2011", "2012"]
recovery_years = ["2013", "2014", "2015", "2016", "2017"]
geo_data_path = "/Users/ameliabaum/Desktop/Amelia/CRA_Thesis/communityreinvestmentact/data/CTs_geo_data_1/"
parsed_data_path = "/Users/ameliabaum/Desktop/Amelia/CRA_Thesis/communityreinvestmentact/data/parsed_data_1/"
shapefiles_data_path = "/Users/ameliabaum/Desktop/Amelia/CRA_Thesis/communityreinvestmentact/data/raw_shapefiles/"
to_all_geodata = "/Users/ameliabaum/Desktop/Amelia/CRA_Thesis/communityreinvestmentact/data/all_bay_geodata/"



## Shapefiles

In [6]:
#2000 census tract shapefiles (from 2008 file, but identical to 2000)
shapefiles_08 = geopandas.read_file(shapefiles_data_path+"alltracts_shapefiles_2008/tl_2008_06_tract00.shp")
bay_shapefiles_08 = shapefiles_08[shapefiles_08["COUNTYFP00"].isin(['001', '013', '041', '055', '075', '081', '085', '097', '095'])]
bay_shapefiles_08["NAME00"] = pd.to_numeric(bay_shapefiles_08["NAME00"]) #1405 tracts
display(bay_shapefiles_08.head())
print(len(bay_shapefiles_08))
# Carolina says that 2010 and should be used for all HMDA data 2012-2017, but for right now I'm still using 
#2000 for these because it seems like the only one that conserves all the data in the merge

shapefiles_10 = geopandas.read_file(shapefiles_data_path+"tl_2010_06_tract10/tl_2010_06_tract10.shp")
bay_shapefiles_10 = shapefiles_10[shapefiles_10["COUNTYFP10"].isin(['001', '013', '041', '055', '075', '081', '085', '097', '095'])]
bay_shapefiles_10["NAME"] = pd.to_numeric(bay_shapefiles_10["NAME10"]) #1588 tracts
display(bay_shapefiles_10.head()) #1588
print(len(bay_shapefiles_10))



/Users/ameliabaum/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  after removing the cwd from sys.path.


,STATEFP00,COUNTYFP00,TRACTCE00,CTIDFP00,NAME00,NAMELSAD00,MTFCC00,FUNCSTAT00,geometry
107,06,055,200501,06055200501,2005.01,Census Tract 2005.01,G5020,S,"POLYGON ((-122.300587 38.300888, -122.300696 3..."
108,06,055,201600,06055201600,2016.00,Census Tract 2016,G5020,S,"POLYGON ((-122.460436 38.514535, -122.459987 3..."
109,06,055,200701,06055200701,2007.01,Census Tract 2007.01,G5020,S,"POLYGON ((-122.318333 38.306618, -122.318355 3..."
110,06,055,201500,06055201500,2015.00,Census Tract 2015,G5020,S,"POLYGON ((-122.547175 38.521858, -122.5482 38...."
111,06,055,201300,06055201300,2013.00,Census Tract 2013,G5020,S,"POLYGON ((-122.356477 38.40573, -122.355529 38..."


1405


/Users/ameliabaum/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if sys.path[0] == '':


,STATEFP10,COUNTYFP10,TRACTCE10,GEOID10,NAME10,NAMELSAD10,MTFCC10,FUNCSTAT10,ALAND10,AWATER10,INTPTLAT10,INTPTLON10,geometry,NAME
921,06,085,509201,06085509201,5092.01,Census Tract 5092.01,G5020,S,1579162,0,+37.4012062,-122.0743461,"POLYGON ((-122.06955 37.4084, -122.069507 37.4...",5092.01
922,06,085,510500,06085510500,5105,Census Tract 5105,G5020,S,2590775,0,+37.3927828,-122.1201120,"POLYGON ((-122.11405 37.38211, -122.114053 37....",5105.00
923,06,085,509401,06085509401,5094.01,Census Tract 5094.01,G5020,S,666430,0,+37.4069811,-122.1144543,"POLYGON ((-122.120053 37.406258, -122.120004 3...",5094.01
924,06,085,509303,06085509303,5093.03,Census Tract 5093.03,G5020,S,573577,0,+37.4060297,-122.0922777,"POLYGON ((-122.09708 37.40364, -122.096921 37....",5093.03
925,06,085,503306,06085503306,5033.06,Census Tract 5033.06,G5020,S,1667716,0,+37.3314079,-121.8221350,"POLYGON ((-121.816822 37.328505, -121.817672 3...",5033.06


1588


## Merging HMDA data with shapefiles for mapping

#### Merge 2009 HMDA data with 2000 (2008) tract definitions

In [13]:
all_counties = []
for county in bay_county_names:
    parsed_df = pd.read_csv(parsed_data_path+county+'_'+"2009"+'_parsed.csv')
    parsed_df["Geoid"] = parsed_df["Geoid"].apply(lambda x: "0"+str(x))
    with_geo = pd.merge(bay_shapefiles_08, parsed_df, how="right", left_on="CTIDFP00", right_on="Geoid") 
    with_geo.dropna(inplace=True)
    all_counties.append(with_geo)
#     folder_path = geo_data_path+county+"_2009"
#     if not os.path.exists(folder_path):
#         os.makedirs(folder_path)
#     print("writing...", county, year)
#     with_geo.to_file(driver = 'ESRI Shapefile', filename=folder_path+"/"+county+"_geoparsed_2009.shp")
concat = pd.concat(all_counties, axis=0)
concat.to_file(driver = 'ESRI Shapefile', filename=to_all_geodata+"/2009_allcounties.shp")

#### Merge 2013-2017 HMDA data with 2000 (2008) tract definitions

In [14]:

for year in recovery_years:
    all_counties = []
    for county in bay_county_names:
        parsed_df = pd.read_csv(parsed_data_path+county+'_'+year+'_parsed.csv')
        parsed_df["Geoid"] = parsed_df["Geoid"].apply(lambda x: "0"+str(x))
        with_geo = pd.merge(bay_shapefiles_10, parsed_df, how="right", left_on="GEOID10", right_on="Geoid") 
        with_geo.dropna(inplace=True)
        all_counties.append(with_geo)

#         folder_path = geo_data_path+county+"_"+year
#         if not os.path.exists(folder_path):
#             os.makedirs(folder_path)
#         print("writing...", county, year)
#         with_geo.to_file(driver = 'ESRI Shapefile', filename=folder_path+"/"+county+"_geoparsed_"+year+".shp")
    concat = pd.concat(all_counties, axis=0)
    concat.to_file(driver = 'ESRI Shapefile', filename=to_all_geodata+year+"_allcounties.shp")

In [ ]:
all_bay_geodata

## Creating the Recovery and Slump dataframes

In [6]:
#creates a combined dataframe of all the years in 2 groups, and averages the 4 proportions in this interval.
# to_concat_recovery = []
# to_concat_slump = []
# all_dfs = []
# files = [f for f in os.listdir(geo_data_path) if f[-3:] == "csv"]
# for name in files:
#     df_year = name.split('_')[2][:4]
#     if df_year in slump_years:
#         df1 = pd.read_csv(geo_data_path+name)
#         all_dfs.append(df1)
#         to_concat_slump.append(df1)
        
#     if df_year in recovery_years:
#         df2 = pd.read_csv(geo_data_path+name)
#         all_dfs.append(df2)
#         to_concat_recovery.append(df2)

# slump = pd.concat(to_concat_slump, axis=0)
# #.groupby(by=["Tract", "tract num", "STATEFP", "COUNTYFP", "TRACTCE", "AFFGEOID", "GEOID", 
# #                                    "NAME", "LSAD", "ALAND", "AWATER", "geometry"]).mean()

# recovery = pd.concat(to_concat_recovery, axis=0)
# #.groupby(by=["Tract", "tract num", "STATEFP", "COUNTYFP", "TRACTCE", "AFFGEOID", "GEOID", 
# #                                    "NAME", "LSAD", "ALAND", "AWATER", "geometry"]).mean()




            
      
    

In [8]:
# slump.to_csv(geo_data_path+"all_tracts_geoparsed_slump.csv", index=False)
# recovery.to_csv(geo_data_path+"all_tracts_geoparsed_recovery.csv", index=False)